# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 59 new papers today
          13 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/12 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2303.11349


extracting tarball to tmp_2303.11349...

 done.
Retrieving document from  https://arxiv.org/e-print/2303.11358


extracting tarball to tmp_2303.11358...

...

extracting tarball to tmp_2303.11948... done.


Found 46 bibliographic references in tmp_2303.11948/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2303.12004


extracting tarball to tmp_2303.12004...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3460: LatexWarning: Found documentclass in tmp_2303.12004/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:361: LatexWarning: Latex injecting: 'Acknowledgements-only' from 'tmp_2303.12004/Acknowledgements-only.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11948-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11948) | **Forming super-Mercuries: The role of stellar abundances**  |
|| <mark>J. Mah</mark>, <mark>B. Bitsch</mark> |
|*Appeared on*| *2023-03-22*|
|*Comments*| *9 pages, 6 figures, accepted for publication in A&A*|
|**Abstract**| Super-Mercuries, rocky exoplanets with bulk iron mass fraction of more than 60 per cent, appear to be preferentially hosted by stars with higher iron mass fraction than the Earth. It is unclear whether these iron-rich planets can form in the disc, or if giant impacts are necessary. Here we investigate the formation of super-Mercuries in their natal protoplanetary discs by taking into account their host stars' abundances (Fe, Mg, Si, S). We employ a disc evolution model which includes the growth, drift, evaporation and recondensation of pebbles to compute the pebble iron mass fraction. The recondensation of outward-drifting iron vapour near the iron evaporation front is the key mechanism that facilitates an increase in the pebble iron mass fraction. We also simulate the growth of planetary seeds around the iron evaporation front using a planet formation model which includes pebble accretion and planet migration, and compute the final composition of the planets. Our simulations are able to reproduce the observed iron compositions of the super-Mercuries provided that all the iron in the disc are locked in pure Fe grains and that the disc viscosity is low. The combined effects of slow orbital migration of planets and long retention time of iron vapour in low-viscosity discs makes it easier to form iron-rich planets. Furthermore, we find that decreasing the stellar Mg/Si ratio results in an increase in the iron mass fraction of the planet due to a reduction in the abundance of Mg2SiO4, which has a very similar condensation temperature as iron, in the disc. Our results thus imply that super-Mercuries are more likely to form around stars with low Mg/Si, in agreement with observational data. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11349-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11349) | **An extremely compact, low-mass post-starburst galaxy at $z=5.2$**  |
|| Victoria Strait, et al. -- incl., <mark>Katriona Gould</mark> |
|*Appeared on*| *2023-03-22*|
|*Comments*| *9 pages, 4 figures, 1 table*|
|**Abstract**| We report the discovery of a low-mass $z=5.200\pm 0.002$ galaxy that is in the process of ceasing its star formation. The galaxy, MACS0417-z5PSB, is multiply imaged with magnification factors $\sim40$ by the galaxy cluster MACS J0417.5-1154, observed as part of the CAnadian NIRISS Unbiased Cluster Survey (CANUCS). Using observations of MACS0417-z5PSB with a JWST/NIRSpec Prism spectrum and NIRCam imaging, we investigate the mechanism responsible for the cessation of star formation of the galaxy, and speculate about possibilities for its future. Using spectrophotometric fitting, we find a remarkably low stellar mass of $\rm{M_*}=4.3\pm^{0.9}_{0.8} \times 10^{7} \rm{M_{\odot}}$, less than 1% of the characteristic stellar mass at $z\sim5$. We measure a de-lensed rest-UV half-light radius in the source plane of $30\pm^{7}_{5}$ pc, and measure a star formation rate from H$\alpha$ of $0.14\pm^{0.17}_{0.12}$ $\rm{M_{\odot}/yr}$. We find that under the assumption of a double power law star formation history, MACS0417-z5PSB has seen a recent rise in star formation, peaking $\sim10-30$ Myr ago and declining precipitously since then. Together, these measurements reveal a low-mass, extremely compact galaxy which is in the process of ceasing star formation. We investigate the possibilities of mechanisms that have led to the cessation of star formation in MACS0417-z5PSB, considering stellar and AGN feedback, and environmental processes. We can likely rule out an AGN and most environmental processes, but leave open the possibility that MACS0417-z5PSB could be a star forming galaxy in the lull of a bursty star formation history. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11358-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11358) | **HD 42477: coupled r modes, g modes and a p mode in an A0Vnne star**  |
|| D. W. Kurtz, et al. -- incl., <mark>U. Lee</mark> |
|*Appeared on*| *2023-03-22*|
|*Comments*| *11 pages, 6 figures, 2 tables, accepted to MNRAS*|
|**Abstract**| Several studies have shown that a number of stars pulsating in p modes lie between the $\beta$ Cep and $\delta$ Sct instability strips in the Hertzsprung-Russell (HR) Diagram. At present, there is no certain understanding of how p~modes can be excited in this $T_{\rm eff}$ range. The goal of this work is to disprove the conjecture that all stars pulsating in p modes and lying in this $T_{\rm eff}$ range are the result of incorrect measurements of $T_{\rm eff}$, contamination, or the presence of unseen cooler companions lying in the $\delta$ Sct instability strip (given the high binary fraction of stars in this region of the HR Diagram). Using TESS data, we show that the A0Vnne star HD 42477 has a single p mode coupled to several r modes and/or g modes. We rule out a contaminating background star with a pixel-by-pixel examination, and we essentially rule out the possibility of a companion $\delta$ Sct star in a binary. We model the pulsations in HD 42477 and suggest that the g modes are excited by overstable convective core modes. We also conjecture that the single p mode is driven by coupling with the g modes, or that the oblateness of this rapidly-rotating star permits driving by He II ionization in the equatorial region. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11847-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11847) | **New evidence for the precession of tilted disk in SDSS  J081256.85+191157.8**  |
|| Qi-Bin Sun, et al. -- incl., <mark>Li-Ying Zhu</mark>, <mark>Er-Gang Zhao</mark>, <mark>Fu-Xing Li</mark>, <mark>Min-Yu Li</mark> |
|*Appeared on*| *2023-03-22*|
|*Comments*| *21 pages, 16 figures*|
|**Abstract**| Super-orbital signals and negative superhumps are thought to be related to the reverse precession of the nodal line in a tilted disk, but the evidence is lacking. Our results provide new evidence for the precession of the tilted disk. Based on the TESS and K2 photometry, we investigate the super-orbital signals, negative superhumps, positive superhumps, and eclipse characteristics of the long-period eclipsing cataclysmic variable star SDSS J0812. We find super-orbital signals, negative superhumps, and positive superhumps with periods of 3.0451(5) d, 0.152047(2) d, and 0.174686(7) d, respectively, in the K2 photometry, but all disappear in the TESS photometry, where the positive superhumps are present only in the first half of the same campaign, confirming that none of them is permanently present in SDSS J0812. In addition, we find for the first time a cyclic variation of the O-C of minima, eclipse depth, and negative superhumps amplitudes for 3.045(8) d, 3.040(6) d, and 3.053(8) d in SDSS J0812, respectively, and all reach the maximum at ~ 0.75 precession phases of the tilted disk, which provides new evidence for the precession of the tilted disk. We suggest that the O-C and eclipse depth variations may come from a shift of the brightness center of the precession tilted disk. Our first finding on the periodic variation of negative superhumps amplitude with the super-orbital signals is significant evidence that the origin of negative superhumps is related to the precession of the tilted disk. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11946-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11946) | **JWST/NIRSpec First Census of Broad-Line AGNs at z=4-7: Detection of 10  Faint AGNs with M_BH~10^6-10^7 M_sun and Their Host Galaxy Properties**  |
|| Yuichi Harikane, et al. -- incl., <mark>Yechi Zhang</mark> |
|*Appeared on*| *2023-03-22*|
|*Comments*| *22 pages, 13 figures, comments welcome*|
|**Abstract**| We present a first statistical sample of faint type-1 AGNs at $z>4$ identified by JWST/NIRSpec deep spectroscopy. Among the 185 galaxies at $z_\mathrm{spec}=3.8-8.9$ confirmed with NIRSpec, our systematic search for broad-line emission reveals 10 type-1 AGNs at $z=4.015-6.936$ whose broad component is only seen in the permitted H$\alpha$ line and not in the forbidden [OIII]$\lambda$5007 line that is detected with greater significance than H$\alpha$. The broad H$\alpha$ line widths of $\mathrm{FWHM}\simeq1000-6000\ \mathrm{km\ s^{-1}}$ suggest that the AGNs have low-mass black holes with $M_\mathrm{BH}\sim10^6-10^7\ M_\odot$, remarkably lower than those of low-luminosity quasars previously identified at $z>4$ with ground-based telescopes. JWST and HST high-resolution images reveal that the majority of them show extended morphologies indicating significant contribution to the total lights from their host galaxies, except for three compact objects two of which show red SEDs, probably in a transition phase from faint AGNs to low luminosity quasars. Careful AGN-host decomposition analyses show that their host's stellar masses are systematically lower than the local relation between the black hole mass and the stellar mass, implying a fast black hole growth consistent with predictions from theoretical simulations. A high fraction of the broad-line AGNs ($\sim5\%$), higher than $z\sim0$, indicates that a number density of such faint AGNs is higher than an extrapolation of the quasar luminosity function, implying a large population of AGNs including type 1 and type 2 in the early universe. Such faint AGNs contribute to cosmic reionization, while the total contribution is not large, up to $\sim50\%$ at $z\sim6$, because of their faint nature. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.12004-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.12004) | **Comparison of Polarized Radiative Transfer Codes used by the EHT  Collaboration**  |
|| Ben S. Prather, et al. -- incl., <mark>Walter Alef</mark> |
|*Appeared on*| *2023-03-22*|
|*Comments*| *Accepted for publication in ApJ*|
|**Abstract**| Interpretation of resolved polarized images of black holes by the Event Horizon Telescope (EHT) requires predictions of the polarized emission observable by an Earth-based instrument for a particular model of the black hole accretion system. Such predictions are generated by general relativistic radiative transfer (GRRT) codes, which integrate the equations of polarized radiative transfer in curved spacetime. A selection of ray-tracing GRRT codes used within the EHT collaboration is evaluated for accuracy and consistency in producing a selection of test images, demonstrating that the various methods and implementations of radiative transfer calculations are highly consistent. When imaging an analytic accretion model, we find that all codes produce images similar within a pixel-wise normalized mean squared error (NMSE) of 0.012 in the worst case. When imaging a snapshot from a cell-based magnetohydrodynamic simulation, we find all test images to be similar within NMSEs of 0.02, 0.04, 0.04, and 0.12 in Stokes I, Q, U , and V respectively. We additionally find the values of several image metrics relevant to published EHT results to be in agreement to much better precision than measurement uncertainties. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11614-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11614) | **Turbulence in Zeeman Measurements from Molecular Clouds**  |
|| Zhuo Cao, <mark>Hua-bai Li</mark> |
|*Appeared on*| *2023-03-22*|
|*Comments*| *accepted by ApJL*|
|**Abstract**| Magnetic fields (B-fields) play an important role in molecular cloud fragmentation and star formation, but are very difficult to detect. The temporal correlation between the field strength (B) and gas density (n) of an isolated cloud has been suggested as an indication of the dynamical importance of B-fields relative to self-gravity. This temporal B-n relation is, however, unobservable. What can be observed using Zeeman measurements are the "spatial B-n relations" from the current plane of the sky. Nevertheless, the temporal B-n relation argument has still been widely used to interpret observations. Here we present the first numerical test of the legitimacy of this interpretation. From a simulation that can reproduce the observed Zeeman spatial B~n^2/3 relation, we found that temporal B-n relations of individual cores bear no resemblance to the spatial B-n relations. This result inspired us to discover that the true mechanism behind the 2/3 index is random turbulence compression instead of symmetrical gravitational contraction. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2303.11875-b31b1b.svg)](https://arxiv.org/abs/arXiv:2303.11875) | **Recommending Low-Cost Compact Space Environment and Space Weather  Effects Sensor Suites for NASA Missions**  |
|| Yihua Zheng, et al. -- incl., <mark>Joseph Minow</mark> |
|*Appeared on*| *2023-03-22*|
|*Comments*| *White paper submitted to Decadal Survey for Solar and Space Physics (Heliophysics) 2024-2033 It can also be accessed here. this http URL*|
|**Abstract**| As miniaturized spacecraft (e.g., cubesats and smallsats) and instrumentation become an increasingly indispensable part of space exploration and scientific investigations, it is important to understand their potential susceptibility to space weather impacts resulting from the sometimes volatile space environment. There are multitude of complexities involved in how space environment interacts with different space hardware/electronics. Measurements of such impacts, however, have been lacking. Therefore, we recommend developing and/or procuring low-cost, low-power consumption, and compact sensor suites (mainly for space weather and impact purposes) and flying them on all future NASA (and U.S in general) missions in order to measure and quantify space weather impacts, in addition to the main instrumentation. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2303.11948.md


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

 as well as Mercury are included for comparison. The grey band and dashed line represent the range of Mg/Si values of super-Mercury host stars and the Sun, respectively. (*fig:planetFe_stellarMgSi*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

201  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
